# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [2]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'C:\BoSSS-Databases\OscillatingDroplet3D'.


In [4]:
//add database manually:
OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\130.83.248.207\ValidationTests\OscillatingDroplet3D'.


## Observing sessions

In [5]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm1_case1_Oh0.1_AnalytInit	01/02/2022 09:59:19	bb0d73d2...
#1: OscillatingDroplet3D	J432k3_arm0_case1_Oh0.1_AnalytInit	01/02/2022 09:52:56	9827eade...
#2: OscillatingDroplet3D	J432k3_arm0_case4_Oh0.1	01/02/2022 09:52:02	1f588a9c...
#3: OscillatingDroplet3D	J432k3_arm0_case4_Oh0.1_AnalytInit	01/02/2022 09:54:32	1720d000...
#4: OscillatingDroplet3D	J432k3_arm1_case1_Oh0.1	01/02/2022 09:55:50	b5bc961f...
#5: OscillatingDroplet3D	J432k3_arm1_case4_Oh0.1	01/02/2022 09:57:53	f63b800b...
#6: OscillatingDroplet3D	J432k3_arm0_case1_Oh0.1	01/02/2022 09:50:44	eb4da990...
#7: OscillatingDroplet3D	J432k3_arm0_case2_Oh0.1	01/02/2022 09:51:10	dcb5d25d...
#8: OscillatingDroplet3D	J432k3_arm1_case4_Oh0.1_AnalytInit	01/02/2022 10:01:48	766d034c...
#9: OscillatingDroplet3D	J432k3_arm0_case3_Oh0.1	01/02/2022 09:51:35	4dd6c2c8...
#10: OscillatingDroplet3D	J432k3_arm0_case5_Oh0.56	01/02/2022 09:52:28	3cfeee2d...
#11: OscillatingDroplet3D	J432k3_arm1_case5_Oh0.56_AnalytInit	0

In [6]:
//wmg.Sessions.Where(s => s.Name.Contains("J432k3_arm0_case2_Oh0.1_AnalytInit"))
//wmg.Sessions[1].Delete(true)

In [50]:
wmg.Sessions[16]

OscillatingDroplet3D	J432k3_arm1_case3_Oh0.1_AnalytInit*	01/02/2022 10:00:56	c5f103a7...

In [36]:
wmg.Sessions[16].GetSessionDirectory()

\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\c5f103a7-766c-42df-94b4-133a0510f684

In [51]:
wmg.Sessions[16].DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Jan02_100030

In [7]:
using System.IO;

In [8]:
/*
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}
*/

In [9]:
/*
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}
*/

In [10]:
// might take some time
/*
foreach(var s in wmg.Sessions) {
    Console.Write(s.Name + ": ");
    if(s.Timesteps.Count() > 0)
        Console.WriteLine(s.Timesteps.Last());
    else 
        Console.WriteLine("no timestep computed/saved");
    Console.WriteLine();
}
*/

In [11]:
//wmg.Sessions.Single(si => si.Name.Contains("case3")).Export().WithSupersampling(2).Do()

## Plotting of Spherical Harmonics

In [23]:
//string[] modes = new string[]{ "mode2", "mode3", "mode4"};
//string[] aPis = new string[] { "aP0", "aP1", "aP2" };
string[] amrS = new string[] { "arm0", "arm1" };
string[] caseS = new string[] { "case1", "case2", "case3", "case4", "case5" };
bool[] ShouldAnaInit = new bool[] { true, false };
bool skipNonPoly = true; // false: plots all Legendgre functions, i.e. indices (*, *)
//                          true:  plots only Legendre ploynomials, i.e. inices (*, 0)

In [24]:
Plot2Ddata[,] PlotTable = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < 5; iRow++) { // loop over cases
for(int iarm = 0; iarm < 2; iarm++) {
for(int iAna = 0; iAna < 2; iAna++) {
    string _amr = amrS[iarm];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTable[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var column in tab) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
       if(column.Key == "time")
          continue;
       if(skipNonPoly) {
           if(!column.Key.Contains(", 0)")) {
               continue;
           }
       } 
        
       if(iarm == 1 && ShouldAnaInit[iAna] == false)
          fmt.DashType = DashTypes.Solid;
       else if(iarm == 1 && ShouldAnaInit[iAna] == true)
          fmt.DashType = DashTypes.Dashed;
       else if(iarm == 0 && ShouldAnaInit[iAna] == false)
          fmt.DashType = DashTypes.Dotted;
       else if(iarm == 0 && ShouldAnaInit[iAna] == true)
          fmt.DashType = DashTypes.DotDotDashed;
       else
          throw new Exception("unknown setting");
       string name = column.Key;
       if(name.Contains(", 0)"))
           fmt.LineWidth = 3;
       if(iarm > 0)
           name = name + "-amr";
       if(ShouldAnaInit[iAna])
           name = name + "-Ainit";
        
        plot.AddDataGroup(name, time, column.Value, fmt);
   }
   
   //plot.ShowLegend = iCol == 2 && iRow == 2;
   plot.ShowLegend = true;
   
   //plot.Title = SI.Name;
    
   if(iarm == 0) {
       PlotTable[iRow,iCol] = plot;
   } else {
       PlotTable[iRow,iCol] = PlotTable[iRow,iCol].Merge(plot);
   }
   
    
   PlotTable[iRow,iCol].XrangeMin = 0;
   PlotTable[iRow,iCol].XrangeMax = 4;
}

if(PlotTable[iRow,iCol] != null) {
    PlotTable[iRow,iCol].ShowLegend = true; //iCol == 0 && iRow == 0;
    PlotTable[iRow,iCol].LegendFont = 10;
    //if(iRow == 0)
    //   PlotTable[iRow,iCol].Title = aPis[iCol];
    if(iCol == 0)
       PlotTable[iRow,iCol].Ylabel = caseS[iRow];
}
}
}
}

J432k3_arm0_case1_Oh0.1_AnalytInit
J432k3_arm0_case1_Oh0.1
J432k3_arm1_case1_Oh0.1_AnalytInit
J432k3_arm1_case1_Oh0.1
J432k3_arm0_case2_Oh0.1_AnalytInit
J432k3_arm0_case2_Oh0.1
J432k3_arm1_case2_Oh0.1_AnalytInit
J432k3_arm1_case2_Oh0.1
J432k3_arm0_case3_Oh0.1_AnalytInit
J432k3_arm0_case3_Oh0.1
J432k3_arm1_case3_Oh0.1_AnalytInit
J432k3_arm1_case3_Oh0.1
J432k3_arm0_case4_Oh0.1_AnalytInit
J432k3_arm0_case4_Oh0.1
J432k3_arm1_case4_Oh0.1_AnalytInit
J432k3_arm1_case4_Oh0.1
J432k3_arm0_case5_Oh0.56_AnalytInit
J432k3_arm0_case5_Oh0.56
J432k3_arm1_case5_Oh0.56_AnalytInit
J432k3_arm1_case5_Oh0.56


### Case 1

In [44]:
PlotTable[0,0].XrangeMin = 0;
PlotTable[0,0].XrangeMax = 4;
PlotTable[0,0].Legend_maxrows = 4;
PlotTable[0,0].LegendAlignment = new [] { "i", "t", "c" };
var gp = PlotTable[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M176.7,52.6 L213.3,52.6 M71.9,239.3 L72.7,238.7 L73.6,238.5 L74.4,238.5 L75.3,238.7 L76.1,239.1
 L77.0,239.6 L77.8,240.3 L78.7,241.1 L79.5,242.1 L80.4,243.2 L81.2,244.3 L82.1,245.6 L82.9,247.0
 L83.8,248.5 L84.6,250.1 L85.5,251.7 L86.3,253.4 L87.2,255.2 L88.0,257.1 L88.9,259.0 L89.7,260.9
 L90.6,262.9 L91.4,264.9 L92.2,266.9 L93.1,268.9 L93.9,270.9 L94.8,273.0 L95.6,275.0 L96.5,277.0
 L97.3,279.0 L98.2,280.9 L99.0,282.8 L99.9,284.7 L100.7,286.5 L101.6,288.3 L102.4,290.0 L103.3,291.7
 L104.1,293.3 L105.0,294.8 L105.8,296.2 L106.7,297.6 L107.5,298.9 L108.4,300.1 L109.2,301.3 L110.0,302.3
 L110.9,303.3 L111.7,304.2 L112.6,305.0 L113.4,305.7 L114.3,306.3 L115.1,306.9 L116.0,307.3 L116.8,307.7
 L117.7,308.0 L118.5,308.2 L119.4,308.3 L120.2,308.4 L121.1,308.3 L121.9,308.2 L122.8,308.1 L123.6,307.8
 L124.5,307.5 L125.3,307.1 L126.2,306.7 L127.0,306.2 L127.9,305.7 L128.7,305.1 L129.5,304.4 L130.4,303.7
 L131.2,303.0 L132.1,302.2 L132.9,301.4 L133.8,300.6 L134.6,299.7 L135.5,298.8 L136.3,297.9 L137.2,296.9
 L138.0,296.0 L138.9,295.0 L139.7,294.0 L140.6,293.1 L141.4,292.1 L142.3,291.1 L143.1,290.1 L144.0,289.1
 L144.8,288.1 L145.7,287.1 L146.5,286.1 L147.3,285.1 L148.2,284.1 L149.0,283.2 L149.9,282.2 L150.7,281.2
 L151.6,280.3 L152.4,279.4 L153.3,278.4 L154.1,277.5 L155.0,276.6 L155.8,275.7 L156.7,274.8 L157.5,273.9
 L158.4,273.0 L159.2,272.2 L160.1,271.4 L160.9,270.5 L161.8,269.7 L162.6,268.9 L163.5,268.1 L164.3,267.3
 L165.2,266.6 L166.0,265.8 L166.8,265.0 L167.7,264.3 L168.5,263.6 L169.4,262.8 L170.2,264.4 L171.1,262.3
 L171.9,261.1 L172.8,260.2 L173.6,259.4 L174.5,258.6 L175.3,257.9 L176.2,257.2 L177.0,256.5 L177.9,255.7
 L178.7,255.0 L179.6,254.3 L180.4,253.6 L181.3,252.9 L182.1,252.2 L183.0,251.5 L183.8,250.8 L184.7,250.0
 L185.5,249.3 L186.3,248.6 L187.2,247.9 L188.0,247.2 L188.9,246.5 L189.7,245.8 L190.6,245.1 L191.4,244.4
 L192.3,243.7 L193.1,243.1 L194.0,242.5 L194.8,241.8 L195.7,241.2 L196.5,240.6 L197.4,240.0 L198.2,239.5
 L199.1,238.9 L199.9,238.4 L200.8,237.9 L201.6,239.3 L202.5,238.2 L203.3,238.4 L204.1,237.4 L205.0,236.6
 L205.8,236.0 L206.7,235.5 L207.5,234.9 L208.4,234.3 L209.2,233.6 L210.1,232.7 L210.9,231.8 L211.8,232.2
 L212.6,230.7 L213.5,229.6 L214.3,228.6 L215.2,227.7 L216.0,226.9 L216.9,226.2 L217.7,225.6 L218.6,225.0
 L219.4,224.5 L220.3,224.1 L221.1,219.2 L222.0,215.5 L222.8,212.8 L223.6,210.7 L224.5,209.1 L225.3,208.0
 L226.2,207.2 L227.0,206.9 L227.9,206.8 L228.7,207.1 L229.6,207.5 L230.4,208.2 L231.3,209.0 L232.1,210.0
 L233.0,211.0 L233.8,212.2 L234.7,213.4 L235.5,214.7 L236.4,216.0 L237.2,217.4 L238.1,218.7 L238.9,220.1
 L239.8,221.5 L240.6,222.9 L241.5,224.3 L242.3,225.7 L243.1,227.1 L244.0,228.5 L244.8,229.8 L245.7,231.2
 L246.5,232.5 L247.4,233.8 L248.2,235.1 L249.1,236.3 L249.9,237.6 L250.8,238.8 L251.6,240.0 L252.5,241.1
 L253.3,242.2 L254.2,243.3 L255.0,244.4 L255.9,245.4 L256.7,246.4 L257.6,247.4 L258.4,248.4 L259.3,249.3
 L260.1,250.2 L260.9,251.0 L261.8,251.9 L262.6,252.7 L263.5,253.4 L264.3,254.2 L265.2,254.9 L266.0,255.5
 L266.9,256.2 L267.7,256.7 L268.6,257.3 L269.4,257.8 L270.3,258.3 L271.1,258.8 L272.0,259.2 L272.8,259.6
 L2

### Case 2

In [42]:
PlotTable[1,0].XrangeMin = 0;
PlotTable[1,0].XrangeMax = 4;
PlotTable[1,0].Legend_maxrows = 4;
PlotTable[1,0].LegendAlignment = new [] { "i", "c", "c" };
var gp = PlotTable[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside center center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case2 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M176.7,277.6 L213.3,277.6 M71.9,231.4 L72.7,231.8 L73.6,231.4 L74.4,230.4 L75.3,228.9 L76.1,248.5
 L77.0,258.5 L77.8,262.4 L78.7,265.1 L79.5,271.6 L80.4,268.9 L81.2,267.6 L82.1,265.7 L82.9,263.4
 L83.8,260.7 L84.6,257.7 L85.5,254.5 L86.3,251.2 L87.2,247.5 L88.0,243.7 L88.9,239.7 L89.7,235.4
 L90.6,230.8 L91.4,226.0 L92.2,220.7 L93.1,214.9 L93.9,208.3 L94.8,200.8 L95.6,193.7 L96.5,187.1
 L97.3,181.0 L98.2,175.3 L99.0,170.3 L99.9,165.3 L100.7,162.5 L101.6,159.5 L102.4,156.9 L103.3,154.0
 L104.1,151.0 L105.0,148.2 L105.8,145.4 L106.7,142.9 L107.5,140.8 L108.4,141.2 L109.2,139.6 L110.0,137.8
 L110.9,135.6 L111.7,133.8 L112.6,132.0 L113.4,130.5 L114.3,129.2 L115.1,128.2 L116.0,127.5 L116.8,127.0
 L117.7,126.8 L118.5,126.9 L119.4,127.2 L120.2,127.7 L121.1,128.5 L121.9,132.0 L122.8,131.5 L123.6,132.2
 L124.5,133.2 L125.3,134.8 L126.2,135.1 L127.0,137.1 L127.9,139.3 L128.7,141.5 L129.5,143.7 L130.4,145.5
 L131.2,146.4 L132.1,147.0 L132.9,150.1 L133.8,150.3 L134.6,152.6 L135.5,153.0 L136.3,152.1 L137.2,153.5
 L138.0,153.4 L138.9,153.0 L139.7,150.9 L140.6,151.4 L141.4,148.7 L142.3,147.5 L143.1,146.4 L144.0,144.9
 L144.8,143.2 L145.7,141.2 L146.5,139.1 L147.3,136.7 L148.2,134.3 L149.0,131.8 L149.9,129.2 L150.7,126.5
 L151.6,123.8 L152.4,121.1 L153.3,118.4 L154.1,116.0 L155.0,113.9 L155.8,112.4 L156.7,111.3 L157.5,110.1
 L158.4,108.9 L159.2,107.8 L160.1,106.6 L160.9,108.9 L161.8,104.2 L162.6,102.8 L163.5,101.7 L164.3,100.5
 L165.2,99.2 L166.0,97.9 L166.8,96.4 L167.7,95.0 L168.5,93.6 L169.4,92.2 L170.2,90.8 L171.1,89.4
 L171.9,88.1 L172.8,86.8 L173.6,85.6 L174.5,84.5 L175.3,83.5 L176.2,82.5 L177.0,81.6 L177.9,80.9
 L178.7,80.2 L179.6,79.6 L180.4,79.2 L181.3,78.8 L182.1,78.5 L183.0,78.4 L183.8,78.4 L184.7,78.5
 L185.5,78.7 L186.3,79.2 L187.2,79.6 L188.0,80.2 L188.9,80.8 L189.7,81.5 L190.6,82.4 L191.4,83.5
 L192.3,84.5 L193.1,85.6 L194.0,86.8 L194.8,88.1 L195.7,89.4 L196.5,90.9 L197.4,92.4 L198.2,93.9
 L199.1,95.5 L199.9,97.2 L200.8,98.9 L201.6,100.6 L202.5,102.5 L203.3,104.3 L204.1,106.2 L205.0,108.0
 L205.8,109.9 L206.7,111.8 L207.5,113.7 L208.4,115.6 L209.2,117.4 L210.1,119.3 L210.9,121.1 L211.8,122.9
 L212.6,124.6 L213.5,126.4 L214.3,125.5 L215.2,126.7 L216.0,128.3 L216.9,129.0 L217.7,132.0 L218.6,134.7
 L219.4,137.0 L220.3,138.8 L221.1,140.4 L222.0,141.8 L222.8,143.3 L223.6,145.0 L224.5,146.7 L225.3,148.4
 L226.2,150.0 L227.0,151.6 L227.9,153.1 L228.7,154.5 L229.6,155.8 L230.4,157.0 L231.3,158.0 L232.1,159.0
 L233.0,159.8 L233.8,160.5 L234.7,158.0 L235.5,158.8 L236.4,158.6 L237.2,159.1 L238.1,158.7 L238.9,158.9
 L239.8,158.4 L240.6,158.4 L241.5,158.4 L242.3,157.4 L243.1,157.2 L244.0,157.1 L244.8,156.0 L245.7,155.7
 L246.5,155.8 L247.4,155.0 L248.2,154.6 L249.1,154.8 L249.9,155.1 L250.8,155.8 L251.6,156.4 L252.5,156.2
 L253.3,156.3 L254.2,157.1 L255.0,157.6 L255.9,157.7 L256.7,156.6 L257.6,156.4 L258.4,156.0 L259.3,156.1
 L260.1,155.9 L260.9,155.5 L261.8,154.4 L262.6,153.8 L263.5,153.0 L264.3,152.7 L265.2,152.2 L266.0,151.5
 L266.9,150.8 L267.7,150.1 L268.6,149.3 L269.4,148.6 L270.3,147.9 L271.1,147.2 L272.0,146.6 L272.8,146.0
 L273.7,145.4 L274.5,144.9 L275.4,144.5 L276.2,144.0 L277.1,143.7 L277.9,143.3 L278.8,143.1 

### Case 3

In [43]:
PlotTable[2,0].XrangeMin = 0;
PlotTable[2,0].XrangeMax = 4;
PlotTable[2,0].Legend_maxrows = 4;
PlotTable[2,0].LegendAlignment = new [] { "i", "t", "c" };
var gp = PlotTable[2,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5 
 
 
 
 
 -1 
 
 
 
 
 -0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case3 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M176.7,52.6 L213.3,52.6 M71.9,545.7 L72.7,545.3 L73.6,544.5 L74.4,543.3 L75.3,541.5 L76.1,539.2
 L77.0,536.4 L77.8,533.2 L78.7,529.2 L79.5,525.1 L80.4,520.6 L81.2,515.5 L82.1,510.2 L82.9,504.6
 L83.8,498.6 L84.6,491.8 L85.5,485.2 L86.3,478.2 L87.2,471.0 L88.0,463.7 L88.9,456.2 L89.7,448.6
 L90.6,440.3 L91.4,432.8 L92.2,425.0 L93.1,417.1 L93.9,409.2 L94.8,401.3 L95.6,393.5 L96.5,385.6
 L97.3,377.8 L98.2,370.1 L99.0,362.4 L99.9,354.9 L100.7,347.5 L101.6,340.2 L102.4,333.1 L103.3,326.2
 L104.1,319.5 L105.0,312.9 L105.8,306.6 L106.7,300.5 L107.5,294.6 L108.4,289.0 L109.2,283.6 L110.0,278.4
 L110.9,273.5 L111.7,268.8 L112.6,264.2 L113.4,259.9 L114.3,255.9 L115.1,252.2 L116.0,248.8 L116.8,245.7
 L117.7,242.8 L118.5,240.2 L119.4,237.8 L120.2,235.6 L121.1,235.3 L121.9,233.2 L122.8,229.8 L123.6,227.2
 L124.5,225.2 L125.3,223.4 L126.2,221.6 L127.0,220.0 L127.9,218.3 L128.7,216.7 L129.5,215.1 L130.4,213.6
 L131.2,212.0 L132.1,210.5 L132.9,209.0 L133.8,207.6 L134.6,206.1 L135.5,204.8 L136.3,203.4 L137.2,202.2
 L138.0,201.4 L138.9,201.3 L139.7,201.8 L140.6,202.5 L141.4,203.0 L142.3,203.4 L143.1,203.6 L144.0,203.8
 L144.8,203.8 L145.7,203.8 L146.5,203.6 L147.3,203.4 L148.2,203.1 L149.0,202.8 L149.9,202.3 L150.7,201.8
 L151.6,201.3 L152.4,208.1 L153.3,204.4 L154.1,201.5 L155.0,199.6 L155.8,198.1 L156.7,196.9 L157.5,205.1
 L158.4,204.9 L159.2,200.2 L160.1,204.2 L160.9,204.1 L161.8,198.5 L162.6,203.6 L163.5,197.7 L164.3,194.0
 L165.2,191.8 L166.0,190.3 L166.8,190.0 L167.7,190.8 L168.5,192.1 L169.4,193.8 L170.2,195.6 L171.1,197.6
 L171.9,199.7 L172.8,201.9 L173.6,204.2 L174.5,206.6 L175.3,209.1 L176.2,211.6 L177.0,214.2 L177.9,216.8
 L178.7,219.4 L179.6,222.0 L180.4,224.7 L181.3,227.3 L182.1,230.0 L183.0,232.6 L183.8,235.2 L184.7,237.9
 L185.5,240.4 L186.3,243.0 L187.2,245.5 L188.0,247.9 L188.9,250.4 L189.7,252.7 L190.6,255.0 L191.4,257.3
 L192.3,259.5 L193.1,261.7 L194.0,263.7 L194.8,265.8 L195.7,267.7 L196.5,269.6 L197.4,271.4 L198.2,273.2
 L199.1,274.8 L199.9,276.4 L200.8,277.9 L201.6,279.4 L202.5,280.8 L203.3,282.1 L204.1,283.3 L205.0,284.5
 L205.8,285.6 L206.7,286.6 L207.5,287.5 L208.4,288.4 L209.2,289.2 L210.1,290.0 L210.9,290.7 L211.8,291.3
 L212.6,291.8 L213.5,292.3 L214.3,294.4 L215.2,294.6 L216.0,293.8 L216.9,293.7 L217.7,293.8 L218.6,293.9
 L219.4,294.1 L220.3,294.2 L221.1,294.2 L222.0,294.2 L222.8,295.6 L223.6,295.3 L224.5,294.4 L225.3,295.4
 L226.2,294.9 L227.0,293.8 L227.9,295.1 L228.7,294.4 L229.6,293.8 L230.4,293.8 L231.3,292.6 L232.1,293.5
 L233.0,292.9 L233.8,292.9 L234.7,292.4 L235.5,292.2 L236.4,290.9 L237.2,290.0 L238.1,289.4 L238.9,288.9
 L239.8,288.5 L240.6,288.0 L241.5,287.6 L242.3,287.1 L243.1,286.6 L244.0,286.2 L244.8,285.7 L245.7,285.2
 L246.5,284.6 L247.4,284.1 L248.2,283.5 L249.1,283.0 L249.9,282.4 L250.8,281.8 L251.6,281.1 L252.5,283.7
 L253.3,282.9 L254.2,280.2 L255.0,281.8 L255.9,281.1 L256.7,278.1 L257.6,279.8 L258.4,279.1 L259.3,275.7
 L260.1,273.9 L260.9,272.7 L261.8,271.7 L262.6,270.7 L263.5,269.8 L264.3,268.9 L265.2,268.1 L266.0,267.2
 L266.9,266.3 L267.7,265.5 L268.6,264.6 L269.4,263.8 L270.3,263.0 L271.1,262.2 L272.0,261.4 L272.8,260.6
 L273.7,259.9 L274.5,259.2 L275.4,258.5 L276.2,257.9 L27

### Case 4

In [37]:
PlotTable[3,0].XrangeMin = 0;
PlotTable[3,0].XrangeMax = 4;
PlotTable[3,0].Legend_maxrows = 4;
PlotTable[3,0].LegendAlignment = new [] { "i", "c", "c" };
var gp = PlotTable[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside center center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case4 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M176.7,277.6 L213.3,277.6 M71.9,116.9 L72.7,116.4 L73.6,116.3 L74.4,116.2 L75.3,116.1 L76.1,116.1
 L77.0,116.1 L77.8,116.1 L78.7,116.2 L79.5,116.2 L80.4,116.4 L81.2,116.5 L82.1,116.7 L82.9,116.9
 L83.8,117.2 L84.6,117.5 L85.5,117.8 L86.3,118.2 L87.2,118.6 L88.0,119.0 L88.9,119.4 L89.7,119.9
 L90.6,120.4 L91.4,120.9 L92.2,121.4 L93.1,122.0 L93.9,122.5 L94.8,123.1 L95.6,123.7 L96.5,124.2
 L97.3,124.8 L98.2,125.4 L99.0,125.9 L99.9,126.5 L100.7,127.0 L101.6,127.5 L102.4,128.0 L103.3,128.6
 L104.1,129.0 L105.0,129.5 L105.8,130.0 L106.7,130.4 L107.5,130.8 L108.4,131.2 L109.2,131.6 L110.0,131.9
 L110.9,132.2 L111.7,132.5 L112.6,132.8 L113.4,135.5 L114.3,133.9 L115.1,133.7 L116.0,133.7 L116.8,133.8
 L117.7,133.9 L118.5,134.0 L119.4,134.0 L120.2,134.1 L121.1,134.1 L121.9,134.1 L122.8,134.1 L123.6,134.1
 L124.5,134.1 L125.3,134.0 L126.2,134.0 L127.0,133.9 L127.9,133.8 L128.7,133.7 L129.5,133.5 L130.4,133.4
 L131.2,133.3 L132.1,133.1 L132.9,132.9 L133.8,132.7 L134.6,132.5 L135.5,132.3 L136.3,132.1 L137.2,131.9
 L138.0,131.6 L138.9,131.4 L139.7,131.1 L140.6,130.8 L141.4,130.5 L142.3,130.3 L143.1,130.0 L144.0,129.7
 L144.8,129.4 L145.7,129.1 L146.5,128.7 L147.3,128.4 L148.2,128.1 L149.0,127.8 L149.9,127.4 L150.7,127.1
 L151.6,126.7 L152.4,126.4 L153.3,126.0 L154.1,125.7 L155.0,125.3 L155.8,125.0 L156.7,124.6 L157.5,124.2
 L158.4,123.8 L159.2,123.5 L160.1,123.1 L160.9,122.7 L161.8,122.3 L162.6,121.9 L163.5,121.5 L164.3,121.1
 L165.2,120.7 L166.0,120.3 L166.8,120.0 L167.7,119.5 L168.5,119.1 L169.4,118.7 L170.2,118.3 L171.1,117.9
 L171.9,117.5 L172.8,117.2 L173.6,116.8 L174.5,116.4 L175.3,116.0 L176.2,115.7 L177.0,115.3 L177.9,115.0
 L178.7,114.6 L179.6,114.3 L180.4,113.9 L181.3,113.6 L182.1,113.2 L183.0,112.8 L183.8,112.4 L184.7,112.1
 L185.5,114.4 L186.3,113.5 L187.2,113.8 L188.0,112.0 L188.9,111.3 L189.7,110.8 L190.6,110.4 L191.4,110.0
 L192.3,109.6 L193.1,109.3 L194.0,108.9 L194.8,111.0 L195.7,109.2 L196.5,108.5 L197.4,108.1 L198.2,107.7
 L199.1,107.4 L199.9,107.1 L200.8,106.9 L201.6,106.7 L202.5,106.5 L203.3,106.2 L204.1,106.0 L205.0,105.8
 L205.8,105.7 L206.7,105.6 L207.5,105.6 L208.4,105.6 L209.2,105.6 L210.1,105.7 L210.9,105.8 L211.8,105.9
 L212.6,106.0 L213.5,106.2 L214.3,106.4 L215.2,106.6 L216.0,106.8 L216.9,106.9 L217.7,107.1 L218.6,107.3
 L219.4,107.6 L220.3,107.9 L221.1,108.2 L222.0,108.6 L222.8,109.0 L223.6,109.4 L224.5,109.8 L225.3,110.2
 L226.2,110.6 L227.0,111.0 L227.9,111.4 L228.7,111.8 L229.6,112.2 L230.4,112.6 L231.3,113.0 L232.1,113.4
 L233.0,113.8 L233.8,114.1 L234.7,114.5 L235.5,114.9 L236.4,115.2 L237.2,115.6 L238.1,116.0 L238.9,116.3
 L239.8,116.6 L240.6,117.0 L241.5,117.3 L242.3,117.6 L243.1,117.9 L244.0,118.2 L244.8,118.5 L245.7,118.8
 L246.5,119.1 L247.4,119.4 L248.2,119.7 L249.1,120.0 L249.9,120.2 L250.8,120.5 L251.6,120.7 L252.5,121.0
 L253.3,121.2 L254.2,121.4 L255.0,121.7 L255.9,121.9 L256.7,122.1 L257.6,122.3 L258.4,122.5 L259.3,122.6
 L260.1,122.8 L260.9,123.0 L261.8,123.1 L262.6,123.3 L263.5,123.4 L264.3,123.5 L265.2,123.6 L266.0,123.7
 L266.9,123.8 L267.7,123.9 L268.6,124.0 L269.4,124.0 L270.3,124.1 L271.1,124.2 L272.0,124.2 L272.8,124.2
 L273.7,124.3 L274.5,124.3 L275.4,124.3 L276.2,124.3 L277.1,124.3 L277.9,124.3 L278.8,124.2 L279.6

### Case 5

In [38]:
PlotTable[4,0].XrangeMin = 0;
PlotTable[4,0].XrangeMax = 2;
PlotTable[4,0].Legend_maxrows = 4;
PlotTable[4,0].LegendAlignment = new [] { "i", "c", "c" };
var gp = PlotTable[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside center center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case5 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M176.7,277.6 L213.3,277.6 M71.9,224.0 L73.6,223.7 L75.3,223.2 L77.0,222.7 L78.7,222.0 L80.4,221.3
 L82.1,220.5 L83.8,219.6 L85.5,218.7 L87.2,217.7 L88.9,216.7 L90.6,215.6 L92.2,214.5 L93.9,213.4
 L95.6,212.2 L97.3,211.1 L99.0,209.9 L100.7,208.7 L102.4,207.5 L104.1,206.2 L105.8,205.0 L107.5,203.7
 L109.2,202.5 L110.9,201.3 L112.6,200.0 L114.3,198.8 L116.0,197.5 L117.7,196.3 L119.4,195.1 L121.1,193.8
 L122.8,192.6 L124.5,191.4 L126.2,190.2 L127.9,189.0 L129.5,187.8 L131.2,186.7 L132.9,185.5 L134.6,184.3
 L136.3,183.2 L138.0,182.1 L139.7,181.0 L141.4,179.8 L143.1,178.7 L144.8,177.7 L146.5,176.6 L148.2,175.5
 L149.9,174.5 L151.6,173.5 L153.3,172.4 L155.0,171.4 L156.7,170.4 L158.4,169.5 L160.1,168.5 L161.8,167.5
 L163.5,166.6 L165.2,165.7 L166.8,164.8 L168.5,163.9 L170.2,163.0 L171.9,162.1 L173.6,161.2 L175.3,160.4
 L177.0,159.6 L178.7,158.7 L180.4,157.9 L182.1,157.1 L183.8,156.3 L185.5,155.6 L187.2,154.8 L188.9,154.0
 L190.6,153.3 L192.3,152.6 L194.0,151.9 L195.7,151.1 L197.4,150.4 L199.1,149.8 L200.8,149.1 L202.5,148.4
 L204.1,147.8 L205.8,147.1 L207.5,146.5 L209.2,145.9 L210.9,145.2 L212.6,144.6 L214.3,144.0 L216.0,143.4
 L217.7,142.9 L219.4,142.3 L221.1,141.7 L222.8,141.2 L224.5,140.6 L226.2,140.1 L227.9,139.6 L229.6,139.1
 L231.3,138.5 L233.0,138.0 L234.7,137.6 L236.4,137.1 L238.1,136.6 L239.8,136.1 L241.5,135.6 L243.1,135.2
 L244.8,134.7 L246.5,134.3 L248.2,133.9 L249.9,133.4 L251.6,133.0 L253.3,132.6 L255.0,132.2 L256.7,131.8
 L258.4,131.4 L260.1,131.0 L261.8,130.6 L263.5,130.3 L265.2,129.9 L266.9,129.5 L268.6,129.2 L270.3,128.8
 L272.0,128.5 L273.7,128.1 L275.4,127.8 L277.1,127.5 L278.8,127.1 L280.4,126.8 L282.1,126.5 L283.8,126.2
 L285.5,125.9 L287.2,125.6 L288.9,125.3 L290.6,125.0 L292.3,124.7 L294.0,124.4 L295.7,124.2 L297.4,123.9
 L299.1,123.6 L300.8,123.3 L302.5,123.1 L304.2,122.8 L305.9,122.6 L307.6,122.3 L309.3,122.1 L311.0,121.8
 L312.7,121.6 L314.4,121.4 L316.1,121.1 L317.7,120.9 L319.4,120.7 L321.1,120.5 L322.8,120.3 L324.5,120.1
 L326.2,119.9 L327.9,119.7 L329.6,119.5 L331.3,119.3 L333.0,119.1 L334.7,118.9 L336.4,118.7 L338.1,118.5
 L339.8,118.3 L341.5,118.1 L343.2,117.9 L344.9,117.8 L346.6,117.6 L348.3,117.4 L350.0,117.3 L351.7,117.1
 L353.4,116.9 L355.0,116.8 L356.7,116.6 L358.4,116.5 L360.1,116.3 L361.8,116.2 L363.5,116.0 L365.2,115.9
 L366.9,115.7 L368.6,115.6 L370.3,115.5 L372.0,115.3 L373.7,115.2 L375.4,115.1 L377.1,114.9 L378.8,114.8
 L380.5,114.7 L382.2,114.5 L383.9,114.4 L385.6,114.3 L387.3,114.1 L389.0,114.0 L390.7,113.9 L392.3,113.8
 L394.0,113.7 L395.7,113.5 L397.4,113.4 L399.1,113.3 L400.8,113.2 L402.5,113.1 L404.2,113.0 L405.9,112.9
 L407.6,112.8 L409.3,112.7 L411.0,112.6 L412.7,112.5 L414.4,112.4 L416.1,112.3 L417.8,112.2 L419.5,112.1
 L421.2,112.1 L422.9,112.0 L424.6,111.9 L426.3,111.8 L428.0,111.8 L429.7,111.7 L431.3,111.6 L433.0,111.5
 L434.7,111.5 L436.4,111.4 L438.1,111.3 L439.8,111.3 L441.5,111.2 L443.2,111.1 L444.9,111.1 L446.6,111.0
 L448.3,110.9 L450.0,110.9 L451.7,110.8 L453.4,110.8 L455.1,110.7 L456.8,110.6 L458.5,110.6 L460.2,110.5
 L461.9,110.5 L463.6,110.4 L465.3,110.3 L467.0,110.3 L468.6,110.2 L470.3,110.2 L472.0,110.1 L473.7,110.1
 L475.4,110.0 L477.1,110.0 L478.8,109.9 L480.5,109.9 L482.2,109.8 L483.9,109.8 L485.6,109.7 L487.3,109.7
 L489.0,109.6 L490.7,109.6 L492.4,109.6 L494.1,109.5 L495.8,109.5 L4

### All-in-one plot

In [48]:
for(int i = 0; i < PlotTable.GetLength(0); i++) {
   PlotTable[i,0].XrangeMin = 0;
   PlotTable[i,0].XrangeMax = 2.5;
}

In [49]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 
set key font ",10" inside center center  maxrows 4 Left reverse 
set key font ",10" inside top center  maxrows 4 Left reverse 
set key font ",10" inside center center  maxrows 4 Left reverse 
set key font ",10" inside center center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M676.6,55.6 L713.2,55.6 M89.8,124.6 L93.1,124.3 L96.4,124.2 L99.7,124.2 L102.9,124.3 L106.2,124.5
 L109.5,124.7 L112.8,125.0 L116.1,125.3 L119.4,125.7 L122.6,126.2 L125.9,126.7 L129.2,127.2 L132.5,127.8
 L135.8,128.4 L139.1,129.1 L142.4,129.8 L145.6,130.5 L148.9,131.3 L152.2,132.0 L155.5,132.8 L158.8,133.6
 L162.1,134.5 L165.3,135.3 L168.6,136.2 L171.9,137.0 L175.2,137.9 L178.5,138.7 L181.8,139.6 L185.1,140.4
 L188.3,141.2 L191.6,142.1 L194.9,142.9 L198.2,143.6 L201.5,144.4 L204.8,145.2 L208.0,145.9 L211.3,146.6
 L214.6,147.2 L217.9,147.9 L221.2,148.5 L224.5,149.1 L227.8,149.6 L231.0,150.1 L234.3,150.6 L237.6,151.1
 L240.9,151.5 L244.2,151.8 L247.5,152.2 L250.7,152.5 L254.0,152.7 L257.3,153.0 L260.6,153.2 L263.9,153.3
 L267.2,153.4 L270.5,153.5 L273.7,153.6 L277.0,153.6 L280.3,153.6 L283.6,153.5 L286.9,153.5 L290.2,153.4
 L293.4,153.2 L296.7,153.1 L300.0,152.9 L303.3,152.7 L306.6,152.5 L309.9,152.2 L313.2,151.9 L316.4,151.6
 L319.7,151.3 L323.0,151.0 L326.3,150.7 L329.6,150.3 L332.9,150.0 L336.1,149.6 L339.4,149.2 L342.7,148.8
 L346.0,148.4 L349.3,148.0 L352.6,147.6 L355.9,147.2 L359.1,146.7 L362.4,146.3 L365.7,145.9 L369.0,145.5
 L372.3,145.1 L375.6,144.6 L378.8,144.2 L382.1,143.8 L385.4,143.4 L388.7,143.0 L392.0,142.6 L395.3,142.2
 L398.6,141.8 L401.8,141.4 L405.1,141.0 L408.4,140.6 L411.7,140.2 L415.0,139.9 L418.3,139.5 L421.5,139.1
 L424.8,138.7 L428.1,138.4 L431.4,138.0 L434.7,137.7 L438.0,137.3 L441.3,137.0 L444.5,136.7 L447.8,136.3
 L451.1,136.0 L454.4,135.7 L457.7,135.4 L461.0,135.1 L464.2,134.8 L467.5,134.5 L470.8,135.1 L474.1,134.2
 L477.4,133.7 L480.7,133.3 L484.0,133.0 L487.2,132.7 L490.5,132.4 L493.8,132.1 L497.1,131.8 L500.4,131.5
 L503.7,131.2 L506.9,130.9 L510.2,130.6 L513.5,130.3 L516.8,130.0 L520.1,129.7 L523.4,129.4 L526.7,129.1
 L529.9,128.8 L533.2,128.5 L536.5,128.2 L539.8,127.9 L543.1,127.6 L546.4,127.3 L549.6,127.0 L552.9,126.7
 L556.2,126.4 L559.5,126.1 L562.8,125.9 L566.1,125.6 L569.4,125.4 L572.6,125.1 L575.9,124.9 L579.2,124.6
 L582.5,124.4 L585.8,124.2 L589.1,124.0 L592.3,124.5 L595.6,124.1 L598.9,124.2 L602.2,123.7 L605.5,123.4
 L608.8,123.2 L612.1,122.9 L615.3,122.7 L618.6,122.4 L621.9,122.1 L625.2,121.8 L628.5,121.4 L631.8,121.6
 L635.0,120.9 L638.3,120.5 L641.6,120.0 L644.9,119.7 L648.2,119.3 L651.5,119.0 L654.8,118.8 L658.0,118.5
 L661.3,118.3 L664.6,118.2 L667.9,116.1 L671.2,114.6 L674.5,113.4 L677.7,112.5 L681.0,111.9 L684.3,111.4
 L687.6,111.1 L690.9,110.9 L694.2,110.9 L697.5,111.0 L700.7,111.2 L704.0,111.5 L707.3,111.8 L710.6,112.2
 L713.9,112.7 L717.2,113.2 L720.4,113.7 L723.7,114.2 L727.0,114.8 L730.3,115.3 L733.6,115.9 L736.9,116.5
 L740.2,117.1 L743.4,117.7 L746.7,118.3 L750.0,118.8 L753.3,119.4 L756.6,120.0 L759.9,120.6 L763.1,121.1
 L766.4,121.7 L769.7,122.2 L773.0,122.8 L776.3,123.3 L779.6,123.8 L782.9,124.3 L786.1,124.8 L789.4,125.3
 L792.7,125.8 L796.0,126.2 L799.3,126.7 L802.6,127.1 L805.8,127.6 L809.1,128.0 L812.4,128.4 L815.7,128.8
 L819.0,129.1 L822.3,129.5 L825.6,129.8 L828.8,130.2 L832.1,130.5 L835.4,130.8 L838.7,131.1 L842.0,131.4
 L845.3,131.6 L848.5,131.9 L851.8,132.1 L855.1,132.3 L858.4,132.6 L861.7,132.7 L865.0,132.9 L868.3,133.1
 L871.5,133.2 L874.8,133.4 L878.1,133.5 L

In [ ]:
//Note: you must create sub-directory "latex" first!
//var cl = gp.PlotCairolatex(xSize:32,ySize:18);
//cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}